### *This file allows to generate all data necessary for all figures*

# **Useful packages and functions**

In [1]:
using DifferentialEquations, Plots, Polynomials, LaTeXStrings, ColorSchemes, DelimitedFiles
using Statistics, StatsPlots, Random, ProgressMeter, Printf, LinearAlgebra
include("FYON_2022_STG_kinetics.jl") # Loading of STG kinetics of gating variables
include("FYON_2022_STG_models.jl") # Loading of STG model
include("FYON_2022_STG_utils.jl") # Loading of some utils functions
include("FYON_2022_STG_gs_derivatives.jl") # Loading of X_inf derivatives
include("FYON_2022_STG_DIC.jl") # Loading of the DIC and compensation algorithm
include("FYON_2022_STG_neuromodulation.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [2]:
# Definition of simulation time (in ms)
const Tfinal = 5000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV) and membrane capacitance
const C     = 1. # Membrane capacitance
const VNa   = 50 # Sodium reversal potential
const VK    = -80 # Potassium reversal potential
const VCa   = 80 # Calcium reversal potential
const VH    = -20 # HCN channels reversal potential
const Vleak = -50 # Leak reversal potential

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200;

# **Generating high frequency bursting neurons using Monte Carlo (Figure 1)**

In [3]:
# Initializing some variables
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02
N = 7e4
VV_maxmax = 49.95
VV_maxmin = 49.9
VV_minmax = -69.
VV_minmin = -76.
nb_spikes_target = 5
burstiness_min = 3000.
burstiness_max = 7000.
interburst_f_min = 8.8
interburst_f_max = 9.9
intraburst_f_max = 140.
intraburst_f_min = 70.;

In [ ]:
# Fixing random seed
Random.seed!(542)

# Initializing a flag
flag_start = true

@showprogress 1 "Computing..." for i = 1 : N
    # Generating random conductances
    gleak = 0.007 + 0.007 * rand(1, 1)[1]
    gCaT = 0.95 * CaT_max * rand(1, 1)[1]
    gCaS = 0.95 * CaS_max * rand(1, 1)[1]
    gKd = 0.95 * Kd_max * rand(1, 1)[1]
    gKCa = 0.95 * KCa_max * rand(1, 1)[1]
    gNa = 0.95 * Na_max * rand(1, 1)[1]
    gA = 0.95 * A_max * rand(1, 1)[1]
    gH = 0.95 * H_max * rand(1, 1)[1]

    # Parameter vector for simulations
    p = [0., gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

    # Initial conditions
    V0  = -70.
    Ca0 = 0.5
    x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
          hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

    # Simulation
    prob = ODEProblem(STG_ODE, x0, tspan, p) # Describing the problem
    sol = solve(prob, saveat=0.1, verbose=false, maxiters=40) # Solving the problem

    # Saving results
    VV = sol[1, 30001:end]
    tt = sol.t[30001:end]

    # Computing spiking frequency
    if length(tt) < 1
        burstiness = 0.
        nb_spikes = 0.
        intraburst_f = 0.
        interburst_f = 0.
        V_maxi = 0.
        V_mini = 0.
    elseif tt[end] != Tfinal
        burstiness = 0.
        nb_spikes = 0.
        intraburst_f = 0.
        interburst_f = 0.
        V_maxi = 0.
        V_mini = 0.
    else
        (burstiness, nb_spikes, intraburst_f, interburst_f) = extract_burstiness(VV, tt)
        V_maxi = maximum(VV)
        V_mini = minimum(VV)
    end
    
    # Saving if spiking frequency is in the right order of magnitude
    if (V_maxi <= VV_maxmax && V_maxi >= VV_maxmin &&
        V_mini <= VV_minmax && V_mini >= VV_minmin &&
        burstiness <= burstiness_max && burstiness >= burstiness_min &&
        intraburst_f <= intraburst_f_max && intraburst_f >= intraburst_f_min &&
        interburst_f <= interburst_f_max && interburst_f >= interburst_f_min &&
        nb_spikes == nb_spikes_target)
        if flag_start
            # Creating the conductance matrix
            global g_all_MC = [gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak]
            flag_start = false
        else
            # Adding a line to the conductance matrix
            g_all_MC = hcat(g_all_MC, [gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak])

            # If we reach 200 neurons, break the loop
            if size(g_all_MC)[2] >= ncells
                break
            end
        end
    end
end

# Transposing the conductance matrix
if @isdefined g_all_MC
    g_all_MC = g_all_MC'
end;

In [ ]:
writedlm("./data/g_all_MC.dat", g_all_MC);

# **Generating full random neurons using Monte Carlo to project into DICs space (Figure 2A)**

In [3]:
# Definition of the number of cells in the random set
const ncells = 5000;

In [4]:
# Fixing random seed
Random.seed!(544);

# Initializing some variables
Istaticth = 0.01
nb_channels = 8
g_all_Vth = zeros(ncells, nb_channels + 1)
ICs_th_Vth = zeros(ncells, 8)
@showprogress 1 "Computing..." for n = 1 : ncells
    gNa = 600 + 1400 * rand(1,1)[1]
    gCaT = 6 * rand(1,1)[1]
    gCaS = 12 * rand(1,1)[1]
    gA = 20 + 110 * rand(1,1)[1]
    gKd = 90 + 30 * rand(1,1)[1]
    gKCa = 20 + 120 * rand(1,1)[1]
    gH = 0.05 + 0.2 * rand(1,1)[1]
    gleak = 0.01

    (ith, iosc, gf, gs, gu, gin, Istatic) = DICs(V, 0., gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak)

    Ca = -0.94 * (gCaT * mCaT_inf(V[ith])^3 * hCaT_inf(V[ith])*(V[ith]-VCa) +
                  gCaS * mCaS_inf(V[ith])^3 * hCaS_inf(V[ith])*(V[ith]-VCa)) + 0.05
    Iapp = (Istaticth - (-gNa * mNa_inf(V[ith])^3 * hNa_inf(V[ith]) * (V[ith]-VNa) -
                          gCaT * mCaT_inf(V[ith])^3 * hCaT_inf(V[ith]) * (V[ith]-VCa) -
                          gCaS * mCaS_inf(V[ith])^3 * hCaS_inf(V[ith]) * (V[ith]-VCa) -
                          gA * mA_inf(V[ith])^3 * hA_inf(V[ith]) * (V[ith]-VK) -
                          gKCa * mKCa_inf(V[ith],Ca)^4 * (V[ith]-VK) -
                          gKd * mKd_inf(V[ith])^4 * (V[ith]-VK) -
                          gH * mH_inf(V[ith]) * (V[ith]-VH) -
                          gleak * (V[ith]-Vleak)))

    (ith, iosc, gf, gs, gu, gin, Istatic) = DICs(V, Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak)

    ICs_th_Vth[n, :] = [V[ith], V[iosc], gf[ith], gs[ith], gs[iosc], gu[ith], gin[ith], Istatic[ith]]
    g_all_Vth[n, :] = [gNa, gCaT, gCaS, gA, gKd, gKCa, gH, gleak, Iapp]
end

Computing... 100%|███████████████████████████████████████| Time: 0:02:509m39mmmm9mmm


In [5]:
writedlm("./data/ICs_th_Vth.dat", ICs_th_Vth)

writedlm("./data/g_all_Vth.dat", g_all_Vth);

# **Generating high frequency bursting neurons using DICs method (Figure 3A), separated and not**

In [6]:
# Definition of the number of cells in the random set
const ncells = 500;

In [7]:
# Fixing random seed
Random.seed!(544);

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(ncells, 5., guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to the same value of gs_th (convention in the codes)
(g_all_bursting, ICs_th_bursting) = neuromodCaSA(ncells, g_all_init, ICs_th_init, -8., guth);

In [8]:
writedlm("./data/g_all_bursting.dat", g_all_bursting);

In [5]:
# Fixing random seed
Random.seed!(544);

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init_DIC, ICs_th_init_DIC) = degeneracy_fixDICs_neuromodDIC(ncells, 5., guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to obtain doublets and bursting
(g_all_bursting_DIC, ICs_th_bursting_DIC) = neuromodCaSA(ncells, g_all_init_DIC, ICs_th_init_DIC, -8., guth);

In [10]:
writedlm("./data/g_all_bursting_DIC.dat", g_all_bursting_DIC);

In [6]:
# Fixing random seed
Random.seed!(544);

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init_leak, ICs_th_init_leak) = degeneracy_fixDICs_neuromodleak(ncells, 5., guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to obtain spiking, doublets and bursting
(g_all_bursting_leak, ICs_th_bursting_leak) = neuromodCaSA(ncells, g_all_init_leak, ICs_th_init_leak, -8., guth);

In [12]:
writedlm("./data/g_all_bursting_leak.dat", g_all_bursting_leak);

# **Generating many high frequency bursting neurons using DICs method (Figure 3B), not separated**

In [15]:
# Definition of the number of cells in the random set
const ncells = 2000;

In [16]:
# Fixing random seed
Random.seed!(544);

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init_many, ICs_th_init_many) = degeneracy_fixDICs_neuromod(ncells, 5., guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to the same value of gs_th (convention in the codes)
(g_all_bursting_many, ICs_th_bursting_many) = neuromodCaSA(ncells, g_all_init_many, ICs_th_init_many, -8., guth);

In [17]:
writedlm("./data/g_all_bursting_many.dat", g_all_bursting_many);

# **Generating neuromodulated neurons using DICs method (Figure 4), separated and not**

In [18]:
# Definition of the number of cells in the random set
const ncells = 500;

In [7]:
(g_all_spiking, ICs_th_spiking) = neuromodCaSA(ncells, g_all_init, ICs_th_init, 5., guth)

(g_all_doublets, ICs_th_doublets) = neuromodCaSA(ncells, g_all_init, ICs_th_init, -2., guth);

In [20]:
writedlm("./data/g_all_spiking.dat", g_all_spiking)

writedlm("./data/g_all_doublets.dat", g_all_doublets);

In [8]:
(g_all_spiking_DIC, ICs_th_spiking_DIC) = neuromodCaSA(ncells, g_all_init_DIC, ICs_th_init_DIC, 5., guth)

(g_all_doublets_DIC, ICs_th_doublets_DIC) = neuromodCaSA(ncells, g_all_init_DIC, ICs_th_init_DIC, -2., guth);

In [27]:
writedlm("./data/g_all_spiking_DIC.dat", g_all_spiking_DIC)

writedlm("./data/g_all_doublets_DIC.dat", g_all_doublets_DIC);

In [9]:
(g_all_spiking_leak, ICs_th_spiking_leak) = neuromodCaSA(ncells, g_all_init_leak, ICs_th_init_leak, 5., guth)

(g_all_doublets_leak, ICs_th_doublets_leak) = neuromodCaSA(ncells, g_all_init_leak, ICs_th_init_leak, -2., guth);

In [29]:
writedlm("./data/g_all_spiking_leak.dat", g_all_spiking_leak)

writedlm("./data/g_all_doublets_leak.dat", g_all_doublets_leak);

# **Computing burstiness of neuromodulated neurons (Figure 4)**

In [30]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_doublets = zeros(ncells)
nb_spikes_doublets = zeros(ncells)
intraburst_f_doublets = zeros(ncells)
interburst_f_doublets = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_doublets[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_doublets[i], nb_spikes_doublets[i], intraburst_f_doublets[i], 
        interburst_f_doublets[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_doublets_plot = deepcopy(burstiness_doublets)
burstiness_doublets = burstiness_doublets[.!isnan.(burstiness_doublets)]
nb_spikes_doublets = nb_spikes_doublets[.!isnan.(nb_spikes_doublets)]
intraburst_f_doublets = intraburst_f_doublets[.!isnan.(intraburst_f_doublets)]
interburst_f_doublets = interburst_f_doublets[.!isnan.(interburst_f_doublets)]

# Initializing variables
burstiness_bursting = zeros(ncells)
nb_spikes_bursting = zeros(ncells)
intraburst_f_bursting = zeros(ncells)
interburst_f_bursting = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_bursting[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_bursting[i], nb_spikes_bursting[i], intraburst_f_bursting[i], 
        interburst_f_bursting[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting = burstiness_bursting[.!isnan.(burstiness_bursting)]
nb_spikes_bursting = nb_spikes_bursting[.!isnan.(nb_spikes_bursting)]
intraburst_f_bursting = intraburst_f_bursting[.!isnan.(intraburst_f_bursting)]
interburst_f_bursting = interburst_f_bursting[.!isnan.(interburst_f_bursting)];

Computing first part... 100%|████████████████████████████| Time: 0:02:14
Computing second part... 100%|███████████████████████████| Time: 0:03:05


In [31]:
writedlm("./data/burstiness_doublets_plot.dat", burstiness_doublets_plot)
writedlm("./data/burstiness_doublets.dat", burstiness_doublets)

writedlm("./data/burstiness_bursting.dat", burstiness_bursting);

In [10]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_doublets_DIC = zeros(ncells)
nb_spikes_doublets_DIC = zeros(ncells)
intraburst_f_doublets_DIC = zeros(ncells)
interburst_f_doublets_DIC = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_doublets_DIC[i,:]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_doublets_DIC[i], nb_spikes_doublets_DIC[i], intraburst_f_doublets_DIC[i], 
        interburst_f_doublets_DIC[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_doublets_DIC_plot = deepcopy(burstiness_doublets_DIC)
burstiness_doublets_DIC = burstiness_doublets_DIC[.!isnan.(burstiness_doublets_DIC)]
nb_spikes_doublets_DIC = nb_spikes_doublets_DIC[.!isnan.(nb_spikes_doublets_DIC)]
intraburst_f_doublets_DIC = intraburst_f_doublets_DIC[.!isnan.(intraburst_f_doublets_DIC)]
interburst_f_doublets_DIC = interburst_f_doublets_DIC[.!isnan.(interburst_f_doublets_DIC)]

# Initializing variables
burstiness_bursting_DIC = zeros(ncells)
nb_spikes_bursting_DIC = zeros(ncells)
intraburst_f_bursting_DIC = zeros(ncells)
interburst_f_bursting_DIC = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_bursting_DIC[i,:]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_bursting_DIC[i], nb_spikes_bursting_DIC[i], intraburst_f_bursting_DIC[i], 
        interburst_f_bursting_DIC[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting_DIC = burstiness_bursting_DIC[.!isnan.(burstiness_bursting_DIC)]
nb_spikes_bursting_DIC = nb_spikes_bursting_DIC[.!isnan.(nb_spikes_bursting_DIC)]
intraburst_f_bursting_DIC = intraburst_f_bursting_DIC[.!isnan.(intraburst_f_bursting_DIC)]
interburst_f_bursting_DIC = interburst_f_bursting_DIC[.!isnan.(interburst_f_bursting_DIC)];

Computing first part... 100%|████████████████████████████| Time: 0:02:23m
Computing second part... 100%|███████████████████████████| Time: 0:03:03


In [11]:
writedlm("./data/burstiness_doublets_DIC_plot.dat", burstiness_doublets_DIC_plot)
writedlm("./data/burstiness_doublets_DIC.dat", burstiness_doublets_DIC)

writedlm("./data/burstiness_bursting_DIC.dat", burstiness_bursting_DIC);

In [12]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_doublets_leak = zeros(ncells)
nb_spikes_doublets_leak = zeros(ncells)
intraburst_f_doublets_leak = zeros(ncells)
interburst_f_doublets_leak = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_doublets_leak[i,:]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_doublets_leak[i], nb_spikes_doublets_leak[i], intraburst_f_doublets_leak[i], 
        interburst_f_doublets_leak[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_doublets_leak_plot = deepcopy(burstiness_doublets_leak)
burstiness_doublets_leak = burstiness_doublets_leak[.!isnan.(burstiness_doublets_leak)]
nb_spikes_doublets_leak = nb_spikes_doublets_leak[.!isnan.(nb_spikes_doublets_leak)]
intraburst_f_doublets_leak = intraburst_f_doublets_leak[.!isnan.(intraburst_f_doublets_leak)]
interburst_f_doublets_leak = interburst_f_doublets_leak[.!isnan.(interburst_f_doublets_leak)]

# Initializing variables
burstiness_bursting_leak = zeros(ncells)
nb_spikes_bursting_leak = zeros(ncells)
intraburst_f_bursting_leak = zeros(ncells)
interburst_f_bursting_leak = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_bursting_leak[i,:]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

  # Initial conditions
  V0  = -70.
  Ca0 = 0.5
  x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
        hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    
  # Simulation
  prob = ODEProblem(STG_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  (burstiness_bursting_leak[i], nb_spikes_bursting_leak[i], intraburst_f_bursting_leak[i], 
        interburst_f_bursting_leak[i]) = extract_burstiness(V_t, t)
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting_leak = burstiness_bursting_leak[.!isnan.(burstiness_bursting_leak)]
nb_spikes_bursting_leak = nb_spikes_bursting_leak[.!isnan.(nb_spikes_bursting_leak)]
intraburst_f_bursting_leak = intraburst_f_bursting_leak[.!isnan.(intraburst_f_bursting_leak)]
interburst_f_bursting_leak = interburst_f_bursting_leak[.!isnan.(interburst_f_bursting_leak)];

Computing first part... 100%|████████████████████████████| Time: 0:01:55
Computing second part... 100%|███████████████████████████| Time: 0:03:11


In [13]:
writedlm("./data/burstiness_doublets_leak_plot.dat", burstiness_doublets_leak_plot)
writedlm("./data/burstiness_doublets_leak.dat", burstiness_doublets_leak)

writedlm("./data/burstiness_bursting_leak.dat", burstiness_bursting_leak);

# **Computing neurons along neuromodulation paths (Figure 5)**

In [16]:
# Fixing random seed
Random.seed!(544)

# Initializing some variables
gsths = 5. : -0.02 : -9.
n = 40
gCaSs = zeros(length(gsths), n)
gAs = zeros(length(gsths), n)
burstiness = zeros(length(gsths), n)

# Input current definition
Iapp = 0.

# Initial firing pattern
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(n, gsths[1], guth, Vth)

# Looping over all neuromodulated states
@showprogress 1 "Computing..." for i = 1 : length(gsths)
    # Neuromodulating the initial set
    (g_all, ICs_th) = neuromodCaSA(n, g_all_init, ICs_th_init, gsths[i], guth)

    gCaSs[i, :] = g_all[:, 3]
    gAs[i, :] = g_all[:, 4]
    
    # Looping over the set to compute the burstiness
    for j = 1 : n
        # Extracting the maximal ion channel conductances
        (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all[j, :]

        # Parameter vector for simulations
        p = [Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

        # Initial conditions
        V0  = -70.
        Ca0 = 0.5
        x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
              hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

        # Simulation
        prob = ODEProblem(STG_ODE, x0, tspan, p)
        sol = solve(prob, saveat=0.1, verbose=false)

        # Removing transient part
        V_t = sol[1, :][5001 : end]
        t = sol.t[5001 : end]

        # Computing the burstiness and burst characteristics of the firing pattern
        if t[end] != Tfinal
            (burstiness[i, j], _, _, _) = (0, NaN, NaN, NaN)
        else
            (burstiness[i, j], _, _, _) = extract_burstiness(V_t, t)
        end

        # If spiking
        if isnan(burstiness[i, j])
            burstiness[i, j] = 0.
        end
    end
end

Computing... 100%|███████████████████████████████████████| Time: 1:55:02


In [17]:
writedlm("./data/gCaSs_paths.dat", gCaSs)

writedlm("./data/gAs_paths.dat", gAs)

writedlm("./data/burstiness_paths.dat", burstiness)

writedlm("./data/g_all_init_paths.dat", g_all_init);

# **Computing high frequency bursting neurons using homogeneous scaling (Figure supp 1)**

In [10]:
# Definition of the number of cells in the random set
const ncells = 200

g_all_MC = readdlm("./data/g_all_MC.dat");

In [11]:
# Number of ion channels in the STG model
nb_channels = 8

# Means on spiking point cloud
mean_MC = zeros(1, nb_channels)
mean!(mean_MC, g_all_MC)

# Defining scaling factors and limits for the plots
s1 = 1.3
s2 = 1.3
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02

# Defining scaling factors for homogeneous scaling
gNa_scaling = mean_MC[1] / mean_MC[8]
gCaT_scaling = mean_MC[2] / mean_MC[8]
gCaS_scaling = mean_MC[3] / mean_MC[8]
gA_scaling = mean_MC[4] / mean_MC[8]
gKCa_scaling = mean_MC[5] / mean_MC[8]
gKd_scaling = mean_MC[6] / mean_MC[8]
gH_scaling = mean_MC[7] / mean_MC[8];

In [12]:
# Fixing random seed
Random.seed!(544)

# Initializing a flag
flag_start = true

for i = 1 : ncells
    # Generating random conductances
    gleak = 0.006 + 0.009 * rand(1, 1)[1]
    gCaT = gleak * gCaT_scaling
    gCaS = gleak * gCaS_scaling
    gKd = gleak * gKd_scaling
    gKCa = gleak * gKCa_scaling
    gNa = gleak * gNa_scaling
    gA = gleak * gA_scaling
    gH = gleak * gH_scaling
    
    if flag_start
        # Creating the conductance matrix
        global g_all_MC_leak = [gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak]
        flag_start = false
    else
        # Adding a line to the conductance matrix
        g_all_MC_leak = hcat(g_all_MC_leak, [gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak])
    end
end

# Transposing the conductance matrix
if @isdefined g_all_MC_leak
    g_all_MC_leak = g_all_MC_leak'
end;

In [13]:
# Input current definition
Iapp = 0. : -0.05 : -8.

# Initializing some variables
f_leak = zeros(length(Iapp), ncells)

@showprogress 1 "Computing..." for (k, Iapp_i) in enumerate(Iapp)
    # Looping over all neurons
    for i = 1 : ncells
        # Retrieving maximal conductances from what had been computed
        (gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_MC_leak[i, :]

        # Parameter vector for simulations
        p = [Iapp_i, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C]

        # Initial conditions
        V0  = -70.
        Ca0 = 0.5
        x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
            hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

        # Simulation
        prob = ODEProblem(STG_ODE, x0, tspan, p) # Describing the problem
        sol = solve(prob, saveat=0.1) # Solving the problem

        # Saving results
        VV = sol[1, 30001:end]
        tt = sol.t[30001:end]

        # Computing spiking frequency
        if tt[end] != Tfinal
            f_leak[k, i] = 0.
        else
            f_leak[k, i] = extract_frequency(VV, tt)
            if isnan(f_leak[k, i])
                f_leak[k, i] = 0.
            end
        end
    end
end

Computing... 100%|███████████████████████████████████████| Time: 1:50:56


In [14]:
# Initializing some variables
rheobase = zeros(ncells)

# Computing the rheobase for each neuon
for i = 1 : ncells
    f_leak_i = f_leak[:, i]
    rheobase[i] = maximum(Iapp[findall(f_leak[:, i] .<= 0.)])
end

In [15]:
writedlm("./data/g_all_MC_leak.dat", g_all_MC_leak)

writedlm("./data/f_leak.dat", f_leak)

writedlm("./data/Iapp.dat", Iapp)

writedlm("./data/rheobase.dat", rheobase);